# 1. Redundant Connection

In this problem, a tree is an undirected graph that is connected and has no cycles.

You are given a graph that started as a tree with n nodes labeled from 1 to n, with one additional edge added. The added edge has two different vertices chosen from 1 to n, and was not an edge that already existed. The graph is represented as an array edges of length n where edges[i] = [ai, bi] indicates that there is an edge between nodes ai and bi in the graph.

Return an edge that can be removed so that the resulting graph is a tree of n nodes. If there are multiple answers, return the answer that occurs last in the input.

**Example 1:**
```
1 --2
|  /
| /
|/
3

Input: edges = [[1,2],[1,3],[2,3]]
Output: [2,3]
```
**Example 2:**
```
2 -- 1 -- 5
|    |
3 -- 4

Input: edges = [[1,2],[2,3],[3,4],[1,4],[1,5]]
Output: [1,4]
``` 

**Constraints:**
```
n == edges.length
3 <= n <= 1000
edges[i].length == 2
1 <= ai < bi <= edges.length
ai != bi
There are no repeated edges.
The given graph is connected.
```

### Approach - I

In [ ]:
class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
                
        arr = [-1] * (len(edges)+1)
        
        for s in edges:
        
            firstParent = s[0]
            first = arr[s[0]]
            while first > 0:
                firstParent = first
                first = arr[first]
            
            secondParent = s[1]
            second = arr[s[1]]
            while second > 0:
                secondParent = second
                second = arr[second]
            
            if firstParent == secondParent:
                return s
            
            if first <= second:
                arr[firstParent] += arr[secondParent]
                arr[secondParent] = firstParent
                arr[s[1]] = firstParent
            else:
                arr[secondParent] += arr[firstParent]
                arr[firstParent] = secondParent
                arr[s[0]] = secondParent

### Approach - II

In [ ]:
class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        par = [i for i in range(len(edges) + 1)]
        rank = [1] * (len(edges) + 1)

        def find(n):
            p = par[n]
            while p != par[p]:
                par[p] = par[par[p]]
                p = par[p]
            return p

        # return False if already unioned
        def union(n1, n2):
            p1, p2 = find(n1), find(n2)

            if p1 == p2:
                return False
            if rank[p1] > rank[p2]:
                par[p2] = p1
                rank[p1] += rank[p2]
            else:
                par[p1] = p2
                rank[p2] += rank[p1]
            return True

        for n1, n2 in edges:
            if not union(n1, n2):
                return [n1, n2]

### Approach - III

In [ ]:
from collections import defaultdict
class UnionFind:
    def __init__(self):
        self.parent = defaultdict(lambda: -1)
        self.rank = defaultdict(lambda: 0)
    
    def findRoot(self, x):
        if self.parent[x] == -1:
            return x
        
        # 找到 root 後順便把 parent 改成指到 root, 加速後續可能的 find
        parentRoot = self.findRoot(self.parent[x])
        self.parent[x] = parentRoot
        return parentRoot
    def union(self, x, y):
        xRoot = self.findRoot(x)
        yRoot = self.findRoot(y)

        if xRoot == yRoot:
            return False

        if self.rank[xRoot] < self.rank[yRoot]:
            self.parent[xRoot] = yRoot
        elif self.rank[xRoot] > self.rank[yRoot]:
            self.parent[yRoot] = xRoot
        else:
            self.parent[xRoot] = yRoot
            self.rank[yRoot] += 1
        return True
from typing import *
class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        """
        IDEA:
        
        Run dfs every time an edge is added to see if a loop is formed

        Time: N edges, run N nodes each time, O(n^2)
        
        """
        def dfsCheckLoop(graph, visited, prev, node):
            if node in visited:
                return True
            visited.add(node)
            for neighbor in graph[node]:
                if prev == neighbor:
                    continue
                if dfsCheckLoop(graph, visited, node, neighbor):
                    return True
            return False

        graph = {}
        for a, b in edges:
            if a not in graph:
                graph[a] = set()
            if b not in graph:
                graph[b] = set()

            # add the edge
            graph[a].add(b)
            graph[b].add(a)

            # check loop after adding the edge
            if dfsCheckLoop(graph, set(), None, a):
                return [a, b]

        return []
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        unionFind = UnionFind()
        for a, b in edges:
            if not unionFind.union(a, b):
                return [a, b]
        
        return -1

### Approach - IV

In [ ]:
class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        n = len(edges) + 1
        parent = [i for i in range(n)]
        rank = [1] * n
        # def find(x):
        #     if parent[x]!= x:
        #         return find(parent[x])
        #     return parent[x]

        def find(x):
            p = parent[x]
            # path compression 
            # when root is not found yet
            while p != parent[p]:
                # parent p = it grandpa
                parent[p] = parent[parent[p]]
                # update this grandpa to p
                p = parent[p]
            return p


        # union by rank
        def union(n1,n2):
            p1, p2 = find(n1),find(n2)
            if p1 == p2: return False
            if rank[p1] > rank[p2]:
                parent[p2] = p1
                rank[p1] += rank[p2]
            else:
                parent[p1] = p2
                rank[p2] += rank[p1]
            # successfully union them, return true
            return True

        for n1,n2 in edges:
            if not union(n1,n2):
                return [n1,n2]